In [4]:
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [5]:
class element():
     def __init__(self, mass, C, T):
            '''
            float mass (g), float thermal conductivity (W/gC), float T (K), derived float heat load (W)
            '''
            self.mass = mass
            self.C = C
            self.T = T
            self.P = 0
            
class link():
    def __init__(self, L, A, k, ends, N=1000):
        ''' 
        float length (m), float area (m^2), function thermal condcutivity of T (W/mdegC), list of element()s connected (2 long), int number of points in T integral
        '''
        self.L = L
        self.A = A
        self.k = k
        self.ends = ends
        
    def calc_heat_flow(self):
        '''
        Adds heat flow from hotter end to colder end's internally stored thermal load P
        '''
        hotter_end = ends[np.argmax([elem.T for elem in self.ends])]
        colder_end = ends[1-np.argmax([elem.T for elem in self.ends])]
        
        Ts = np.linspace(colder_end.T, hotter_end.T, N)
        dT = np.diff(Ts)[0]
        ks = self.k(Ts)
        int_k = np.sum(ks) * dT
        
        colder_end.P += self.A / self.L * int_k

In [6]:
def tube_A(outer_r, inner_r):
    '''
    Cross-sec. area of a tube with outer and inner radii.
    '''
    
    outer_A = np.pi*outer_r**2
    inner_A = np.pi*inner_r**2
    
    return outer_A - inner_A

In [7]:
def G10_k(T, params):
    '''
    From NIST.
    '''
    terms = np.array([(np.log10(T))**i for i in range(len(params))])
    exponent = (terms.T * params).T
    return np.power(10, np.sum(exponent, axis=0))

# good down to 10 K
G10_params_norm = np.array([-4.1236, 13.788, -26.068, 26.272, -14.663, 4.4954, -0.6905, 0.0397])
# below is what we use. Good to 12 K.
G10_params_warp = np.array([-2.64827, 8.80228, -24.8998, 41.1625, -39.8754, 23.1778, -7.95635, 1.48806, -0.11701])

In [ ]:
room_wedge_stage = element(1000,  1, 300)
fiftyK_wedge_stage = element(1000,  1, 10)

weddingcake_room_to_50 = link(0.3, 6*tube_A(0.05, 0.05-0.001)